In [2]:
import pandas as pd
from pyconsolida.budget_reader import read_full_budget
from pyconsolida.aggregations import read_all_valid_budgets, load_loop_and_concat
from pyconsolida.delta import get_tabellone_delta
from pathlib import Path

# file_path = Path("/myshare/cantieri/2024/06_Giugno/1496/Analisi_MT1.xlsm")  # Path("/Users/vigji/Downloads/1496/Analisi_MT1_fixed.xlsm")
file_path = Path("/Users/vigji/Downloads/2024/06_Giugno/1496/Analisi_MT1.xlsm")

da_df = read_full_budget(file_path, cache=False)

df_list_sum = read_full_budget(file_path, cache=False, sum_fasi=True)[0]
df_list_sum.loc[641, :]

codice                 641
tipologia       Manodopera
voce          Operaio Icop
costo u.              28.0
u.m.                     h
quantita           21690.0
imp. unit.           952.0
imp.comp.         607320.0
Name: 641, dtype: object

In [3]:
da_file_path = Path("/Users/vigji/Downloads/2024/06_Giugno/1496/Analisi_MT1.xlsm")
a_file_path = Path("/Users/vigji/Downloads/2023/12_Dicembre/1496/Analisi_MT1.xlsm")
da_df, rep = read_all_valid_budgets(
            da_file_path.parent, [da_file_path.parent], tipologie_skip=None, cache=False
        )
a_df, rep = read_all_valid_budgets(
            a_file_path.parent, [a_file_path.parent], tipologie_skip=None, cache=False
        )

# da_df.loc[budget["codice"] == 641, "quantita"].sum()
key_sequence = [
    "commessa",
    "fase",
    "anno",
    "mese",
    "data",
    "mesi-da-inizio",
    "codice",
    "tipologia",
    "voce",
    "costo u.",
    "u.m.",
    "quantita",
    "imp. unit.",
    "imp.comp.",
    "file-hash",
]

In [4]:
from datetime import datetime
def _date_from_string(string):
    month, year = string.split(".")
    return datetime(year=int(year), month=int(month), day=1)

t_start = _date_from_string("12.2023")
t_stop = _date_from_string("06.2024")

In [11]:
def _sum_repetitive_rows(input_df):
    new_index = ["commessa", "codice", "fase"]
    columns_to_sum = ["quantita", "imp.comp."]

    other_cols = list(set(input_df.columns) - set(new_index + columns_to_sum))

        # ensure we first sum together all rows with identical ["commessa", "codice", "fase"] for columns 
    # supporting summing, and we leave the rest as is - it will be filled with the first row
    df_summed = input_df.loc[:, new_index + columns_to_sum]
    df_summed = df_summed.groupby(new_index).sum()

    # for the other columns, we just take the first row:
    df_firstrow = input_df.loc[:, new_index + other_cols]
    df_firstrow = df_firstrow.groupby(new_index).first()
    return pd.concat([df_summed, df_firstrow], axis=1)

def get_tabellone_delta(tabellone_df, t_start_date, t_stop_date):
    """Date due date di inizio e di fine e il tabellone con tutto il dataset, ritorna
    il delta di costi tra le due date.

    Parameters
    ----------
    tabellone : pd.DataFrame
        Il tabellone con tutti i dati.
    t_start_date : datetime
        Data di inizio.
    t_stop_date : datetime
        Data di fine.

    Returns
    -------
    pd.DataFrame
        Il delta di costi tra le due date.
    """
    tabellone_df["data"] = tabellone_df["data"].apply(lambda x: x + "-01")
    tabellone_df["data"] = pd.to_datetime(tabellone_df["data"])

    in_range = tabellone_df[
        (tabellone_df["data"] >= t_start_date) & (tabellone_df["data"] <= t_stop_date)
    ]

    min_dates = in_range.groupby("commessa")["data"].min()
    max_dates = in_range.groupby("commessa")["data"].max()


    start_df = in_range[in_range["data"] == in_range["commessa"].map(min_dates)]
    # Ensure we do not subtract eg march 2024 if we are computing delta since december 2023
    # If beginning does not match t_start, the actual starting point is 0: 
    start_df = start_df[start_df["data"] == t_start_date]

    start_df = _sum_repetitive_rows(start_df)


    end_df = in_range[in_range["data"] == in_range["commessa"].map(max_dates)]
    end_df = _sum_repetitive_rows(end_df)

    # Align:
    start_al_df, end_al_df = start_df.align(end_df)

    # Prepare infos per tabellone:
    line_info_df = end_al_df.loc[:, ["tipologia", "voce", "u.m.", "costo u."]]
    # Trova indice campi rimasti nan:
    missing_info_idx = line_info_df[
        line_info_df["tipologia"].apply(lambda x: type(x) is not str)
    ].index
    line_info_df.loc[missing_info_idx, :] = start_al_df.loc[
        missing_info_idx, ["tipologia", "voce", "u.m.", "costo u."]
    ]

    # Prepara e calcola delta:
    delta_cols = ["quantita", "imp.comp."]
    all_data_cols = (
        [
            "data",
        ]
        + delta_cols
        + [
            "file-hash",
        ]
    )
    da_df = start_al_df.loc[:, all_data_cols]
    a_df = end_al_df.loc[:, all_data_cols]

    delta_df = a_df.loc[:, delta_cols].fillna(0) - da_df.loc[:, delta_cols].fillna(0)

    a_df.columns = [f"A: {col}" for col in a_df.columns]
    da_df.columns = [f"DA: {col}" for col in da_df.columns]
    delta_df.columns = [f"DELTA: {col}" for col in delta_df.columns]

    # Target schema
    # tipologia, voce, u.m., costo u., DA:data, quantità, imp.comp; A - data, quantità, imp.com,; DELTA quantità, imp, a hash, da hash

    final_df = pd.concat([line_info_df, da_df, a_df, delta_df], axis=1)
    final_df = final_df[
        [col for col in final_df.columns if "hash" not in col]
        + ["DA: file-hash", "A: file-hash"]
    ]

    return final_df.reset_index()

budget, reports = load_loop_and_concat(
    [da_file_path.parent, a_file_path.parent],
    key_sequence,
    tipologie_fix=None,
    tipologie_skip=None,
    progress_bar=False,
    report_filename=str(da_file_path.parent.parent.parent.parent / f"report_fixed_tipologie.xlsx"),
    cache=True,
)

delta_df = get_tabellone_delta(budget, t_start, t_stop)

print(delta_df.loc[delta_df["codice"] == 641, "A: quantita"].sum())
delta_df[delta_df["codice"] == 641].head()

21690.0


,commessa,codice,fase,tipologia,voce,u.m.,costo u.,DA: data,DA: quantita,DA: imp.comp.,A: data,A: quantita,A: imp.comp.,DELTA: quantita,DELTA: imp.comp.,DA: file-hash,A: file-hash
269,1496,641,Fase1.10,Personale,Operaio Icop,h,28.0,2023-12-01,1938.0,54264.0,2024-06-01,2289.0,64092.0,351.0,9828.0,6b53429889,c2848368f0
270,1496,641,Fase1.11,Personale,Operaio Icop,h,28.0,2023-12-01,1141.0,31948.0,2024-06-01,1141.0,31948.0,0.0,0.0,6b53429889,c2848368f0
271,1496,641,Fase1.12,Personale,Operaio Icop,h,28.0,2023-12-01,122.0,3416.0,2024-06-01,7626.5,213542.0,7504.5,210126.0,6b53429889,c2848368f0
272,1496,641,Fase1.13,Personale,Operaio Icop,h,28.0,NaT,NaN,NaN,2024-06-01,1786.0,50008.0,1786.0,50008.0,NaN,c2848368f0
273,1496,641,Fase1.14,Personale,Operaio Icop,h,28.0,NaT,NaN,NaN,2024-06-01,1297.0,36316.0,1297.0,36316.0,NaN,c2848368f0


In [6]:
delta_df

NameError: name 'delta_df' is not defined

In [ ]:
delta_df = get_tabellone_delta(budget, t_start, t_stop)


In [39]:
tabellone_path = Path("/Users/vigji/Downloads/exported_da2023-12-01-a-2024-06-01_240830-082458/240830-082458_delta_tabellone.xlsx")

tabellone_df = pd.read_excel(tabellone_path)

filt_df = tabellone_df[(tabellone_df["commessa"] == "1496") & (tabellone_df["codice"] == 641)]

In [45]:
filt_df["A: quantita"]

11182    2289.0
11183    1141.0
11184    7626.5
11185    1736.0
11186      50.0
11187    1297.0
11188      96.0
11189     129.5
11190      95.0
11191      78.0
11192      83.0
11193    1361.0
11194     288.0
11195     136.0
11196      30.0
11197     159.0
11198    1361.0
11199     288.0
11200     136.0
11201      30.0
11202     159.0
11203    1361.0
11204     288.0
11205     136.0
11206      30.0
11207     159.0
11208      30.0
11209    1128.0
11210     265.0
11211     490.0
11212     728.0
11213     520.0
11214    1298.0
11215     163.0
11216     303.0
11217     170.0
Name: A: quantita, dtype: float64

In [33]:
tabellone_df["commessa"].unique()

array(['1434', '1436', '1438', '1441', '1450', '1454', '1455', '1461',
       '1462', '1468', '1470', '1473', '1476', '1482', '1487', '1488',
       '1490', '1492', '1493', '1494', '1496', '1498', '1498-ati', '1499',
       '1501', '1502', '1505', '1506', '1507', '1508', '1512', '1514',
       '1517', '1517b', '1518', '1522', '1524', '1525', '3072', '3081',
       '3091', '3092', '5008', '5009', '5018', '5027', '5028', '9991',
       '9997'], dtype=object)

In [2]:
import pandas as pd
pickle_file = "/Users/vigji/Desktop/Dummy_cantieri/exports/exported_da2023-12-01-a-2024-06-01_240829-173117/240829-173117_tabellone.pickle"
df = pd.read_pickle(pickle_file)

df

,commessa,fase,anno,mese,data,mesi-da-inizio,codice,tipologia,voce,costo u.,u.m.,quantita,imp. unit.,imp.comp.,file-hash
0,1434,Demolizioni,2023,12,2023-12,0,3994,Noli esterni - prestazioni,Strip out (SID),1.0,cad,220492.75,0.935204,220492.75,7a952c78fa
1,1434,Demolizioni,2023,12,2023-12,0,3995,Noli esterni - prestazioni,Demolizioni (SID),1.0,cad,1560710.58,6.619641,1560710.58,7a952c78fa
2,1434,Demolizioni,2023,12,2023-12,0,3996,Noli esterni - prestazioni,Trattamento (SID),1.0,cad,556891.125,2.362014,556891.125,7a952c78fa
3,1434,Demolizioni,2023,12,2023-12,0,3273,Noli esterni - prestazioni,Taglio e riduzione a rottame cisterna in accia...,2500.0,corpo,1.0,0.010604,2500.0,7a952c78fa
4,1434,Demolizioni,2023,12,2023-12,0,3205,Noli esterni - prestazioni,Pulizia e aspirazione carbone (CALCINA),220.0,ora,129.0,0.120372,28380.0,7a952c78fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64541,9991,Fase 220,2024,7,2024-07,7,375,Materiali d'opera,Sabbia 0-4mm,395.42,m3,19.771,20.0,395.42,14ab2b9541
64542,9991,Fase 220,2024,7,2024-07,7,703,Materiali di consumo,Materiale vario di consumo,1502.2,cad,1502.2,1.0,1502.2,14ab2b9541
64543,9991,Fase 220,2024,7,2024-07,7,721,Materiali di consumo,Tavole prismate,1500.0,mc,5.0,300.0,1500.0,14ab2b9541
64544,9991,Fase 220,2024,7,2024-07,7,715,Materiali di consumo,Posa ferro platee-solette FERROBETON,150.0,cad,1.0,150.0,150.0,14ab2b9541


In [16]:
t_start_date = pd.to_datetime("2023-12-01")
t_stop_date = pd.to_datetime("2024-06-01")

tabellone_df = df.copy()
tabellone_df["data"] = tabellone_df["data"].apply(lambda x: x + "-01")
tabellone_df["data"] = pd.to_datetime(tabellone_df["data"])

in_range = tabellone_df[
        (tabellone_df["data"] >= t_start_date) & (tabellone_df["data"] <= t_stop_date)
    ]

min_dates = in_range.groupby("commessa")["data"].min()
max_dates = in_range.groupby("commessa")["data"].max()

new_index = ["commessa", "codice", "fase"]

In [19]:
start_df = in_range[
        in_range["data"] == in_range["commessa"].map(min_dates)
    ]# .set_index(new_index)
start_df = start_df
end_df = in_range[
        in_range["data"] == in_range["commessa"].map(max_dates)
    ].set_index(new_index)

In [6]:
a = 1

In [1]:
from pyconsolida.budget_reader import *
from pyconsolida.budget_reader import _get_valid_costo_rows, _read_raw_budget_sheet
from pathlib import Path
import pandas as pd
import flammkuchen as fl
import numpy as np

## Compute difference

In [14]:
dest_dir = Path("/Users/vigji/Desktop/icop/exported_luigi_sum-fasi-False")

tstamp = "231115_030541"

In [15]:
data_dict = fl.load(dest_dir / f"{tstamp}_python_data.h5")
numerical = ["quantita", "imp. unit."]
budgets_tot, budgets_dec = data_dict["budgets"], data_dict["budgets_dec"]

In [16]:
budgets_tot

,commessa,fase,codice,tipologia,voce,costo u.,u.m.,quantita,imp. unit.,imp.comp.
0,1468,Spese Generali,620,Manodopera,Direttore microtunnel,500.0,g,10.5,51.980198,5250.0
1,1468,Spese Generali,621,Manodopera,Project Manager,350.0,g,104.0,360.39604,36400.0
2,1468,Spese Generali,622,Manodopera,Construction Manager,300.0,g,85.5,253.960396,25650.0
3,1468,Spese Generali,623,Manodopera,ASPP,300.0,g,46.0,136.633663,13800.0
4,1468,Spese Generali,625,Manodopera,Impiegato sede (Bergamasco),200.0,g,18.5,36.633663,3700.0
...,...,...,...,...,...,...,...,...,...,...
16121,1504,Fase2,1106,Materiali di consumo,Gasolio,11.83,l,3996.0,40.390191,47272.68
16122,1504,Fase2,1108,Materiali di consumo,"Materiale vario di consumo (elettrodi, ecc.)",745.0,cad,5.0,3.182673,3725.0
16123,1504,Fase2,1103,Materiali di consumo,Picchi utensili,195.5625,cad,5.0,0.835452,977.8125
16124,1504,Fase2,1116,Materiali di consumo,Tappi per CFA,52.15,cad,64.0,2.851675,3337.6


In [6]:
data_dict = fl.load(dest_dir / f"{tstamp}_python_data.h5")
numerical = ["quantita", "imp. unit."]
budgets_tot, budgets_dec = data_dict["budgets"], data_dict["budgets_dec"]

budgets_tot = budgets_tot.set_index(["commessa", "codice", "fase"]).drop(["costo u.", "imp.comp."], axis=1)
budgets_dec = budgets_dec.set_index(["commessa", "codice", "fase"]).drop(["costo u.", "imp.comp."], axis=1)

dec_al, tot_al = budgets_dec.align(budgets_tot)
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

deltas = tot_al["quantita"] - dec_al["quantita"]
deltas.to_excel(dest_dir / f"{tstamp}_deltas.xlsx")
deltas[deltas < 0].to_excel(dest_dir / f"{tstamp}_negative-deltas.xlsx")


print("Before dropping weird negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

problematic = dec_al[(dec_al["quantita"] > tot_al["quantita"]) & (tot_al["quantita"] > 0)].index

Before dropping weird negatives:  14974437.870095383


In [ ]:
tot_al.loc[problematic, :]

In [ ]:


dec_al = dec_al.drop(problematic)
tot_al = tot_al.drop(problematic)

print("before dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())
negative_correct = dec_al[(dec_al["quantita"] > tot_al["quantita"])].index

dec_al = dec_al.drop(negative_correct)
tot_al = tot_al.drop(negative_correct)

print("dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

In [ ]:
dec_al, tot_al = budgets_dec.align(budgets_tot)
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

In [ ]:
problematic = dec_al[(dec_al["quantita"] > tot_al["quantita"]) & (tot_al["quantita"] > 0)].index

In [ ]:
dec_al.loc[problematic, :]

In [ ]:
tot_al.loc[problematic, :]

In [ ]:
dec_al, tot_al = budgets_dec.align(budgets_tot)
all_diffs = dec_al["quantita"] - tot_al["quantita"]

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)
budgets_dec.drop(list(negative_diffs - not_matching_2021))

dec_al, tot_al = budgets_dec.align(budgets_tot)
numerical = ["quantita"]
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)

tot_al = tot_al.drop(list(negative_diffs))
dec_al = dec_al.drop(list(negative_diffs))

In [ ]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

In [ ]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

In [ ]:
all_diffs = dec_al - 

In [ ]:
s = budgets_dec.loc[list(negative_diffs - not_matching_2021)]

In [ ]:
print("December 2021")
budgets_dec.drop(list(negative_diffs - not_matching_2021))

In [ ]:
print("Sum 2022")
budgets_tot.loc[list(negative_diffs - not_matching_2021)].head()

In [ ]:
all_diffs = tot_al[numerical] - dec_al[numerical]

In [ ]:
all_diffs["quantita"].sum()

In [ ]:
 17930866.01